In [5]:
%%javascript
// Run for table of contents.
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

// https://github.com/kmahelona/ipython_notebook_goodies

<IPython.core.display.Javascript object>

# WMD implementation in Gensim

<h2 id="tocheading">Table of Contents</h2>
<div id="toc"></div>

Note: run JavaScript cell in beginning to get table of contents.


This report covers the implementation of WMD (Word Mover's Distance) in Gensim. It is being worked on at my [word_movers_distance](https://github.com/olavurmortensen/gensim/tree/word_movers_distance) branch of Gensim, and has a pull-request at [#521](https://github.com/piskvorky/gensim/pull/521).

The WMD tutorial on my Gensim WMD branch can be found [here](https://github.com/olavurmortensen/gensim/blob/word_movers_distance/docs/notebooks/WMD_tutorial.ipynb). **Read this tutorial to get an understanding of how WMD works and how to use it in Gensim.**

To understand the model properly, it is also necessary to read the paper, written by Matt J. Kusner, titled ["From Word Embeddings To Document Distances"](http://jmlr.org/proceedings/papers/v37/kusnerb15.pdf).

This pull-request:
* adds one function to `gensim.models.word2vec`, called `wmdistance`.
* adds a class to `gensim.similarities.docsim` called `WmdSimilarity`.

Unit tests of `wmdistance` are added to `gensim.test.test_word2vec`, and unit tests of `WmdSimilarity` are added to `gensim.test.test_similarities`.

A release of WMD has been worked on at the pull-request [#619](https://github.com/piskvorky/gensim/pull/619). #619 is missing some features that are in #521, which are not ready yet.


## pyemd

To compute the WMD distance, we use a Python wrapper of a C++ implementation of EMD (Earth Mover's Distance) called [pyemd](https://github.com/wmayner/pyemd). 

In `gensim.models.word2vec`, a flag `PYEMD_EXT` flag is set to `True` if `emd` is successfully imported from `pemd`. Otherwise, it sets it to `False`, and if WMD is attempted to be used an error is raised.

## The wmdistance function

First, all words that are not in the Word2Vec model's vocabulary are removed, as we cannot compute the distances to such words.

If any of the documents are empty after OOV word removal, we cannot compute the distance. In this case `float('inf')` is returned.

We make a dictionary of the documents using the `gensim.corpora.Dictionary` object.

### Distance matrix

To compute the WMD distance, we first compute the distance between the words in each input document, i.e. the distance matrix. The euclidean distance between the word2vec embeddings of the words in the documents are used.

Only the distances that we need are computed, i.e. distances from document 1 to document 2. All other distances are zero.

### nBOW

nBOW is the normalize bag-of-words representation. We compute the frequency of each word in the document, and normalize these frequencies by the length of the document. The indeces in the nBOW array correspond to the words in the `gensim.corpora.Dictionary` object.

### Computing the WMD

To compute the WMD, we simply pass the nBOW representation of the documents and the distance matrix to `pyemd.emd`, which solves the optimization problem for us.

### WCD

We do *not* need the distance matrix to compute the WCD distance. We do however need the nBOW vectors. To compute the WCD, we stack all the word embeddings (i.e. the Word2Vec vectors) in a matrix $X$, and compute $||X d_1 - X d_2||_2$, where $d_1$ and $d_2$ are the nBOW vectors.

### RWMD

To compute RWMD, we *do* need the distance matrix. We compute the minimal values in the distance matrix, row-wise and column-wise. We then dot those minimum vectors with the nBOW vectors, getting two scalars $RWMD_1$ and $RWMD_2$. We then return $RWMD = \max(RWMD_1, RWMD_2)$.

When we compute RWMD, the indeces in the distance matrix that correspond to pairs of words that do not exist are set to NaN. This is done so that we can use `numpy.nanmin` to find the minimal values that do *not* correspond to word pairs that we do not have in the input doduments. In other words, if we do not do this, the minimum of the distance matrix just returns zeros. There are other ways of dealing with this (for example using "masked arrays").

## The WmdSimilarity class

The `WmdSimilarity` class extends the `SimilarityABC` interface in Gensim. It might be necessary to understand this interface to fully understand the `WmdSimilarity` implementation.

For `WmdSimilarity` to work as a similarity class, where we can use the syntax:
```
# Given a document collection "corpus", train word2vec model.
model = word2vec(corpus)
instance = WmdSimilarity(corpus, model, num_best=10)

# Make query.
sims = instance[query]
```

To be able to do this, it needs to implement a function `get_similarities` that takes a "query", calculates the similarity between the query and each document in the corpus, and returns a vector of similarities.

`get_similarities` also needs to handle a list of queries. 

### Prefetch and prune

if `pp = True`, the prefetch and prune algorithm is used. At the moment, the prefetch and prune algorithm is rather long and confusing, perhaps this can be fixed at some point.

The algorithm works as in the paper [1], that is,

* Compute WCD distance to all documents in corpus.
* Sort according to WCD.
* Compute WMD distances of the first $k$ documents.
* For the rest of the $N - k$ documents:
    * Compute RWMD of current document.
    * If RWMD is greater than WMD of the $k$th document, *continue* (i.e. "prune").
    * Else, compute WMD of current document.
        * If it is greater than WMD of the $k$th document, *continue*.
        * Else, add it to the $k$ nearest documents.

However, there are some caveats.

We are not interested in documents that are identical to the input document. This is how the `SimilarityABC` interface works when `num_best` is not `None`, and it is never `None` when prefetch and prune is being used. To avoid this problem we do two things: (1) if WCD is zero at the start of the algorithm, set it to `float('inf')`, (2) if WMD of a document is zero (during prune process), ignore it and continue.

The `SimilarityABC` interface expects a list of similarities between the query and *all* the documents in the corpus, no just the $k$th (i.e. `num_best`) most similar. So we simply set the closest to their actual WMD distances, and the rest to some large number, such that they are ignored in the sorting process.

## Unit tests

### Testing wmdistance

The unit tests for `wmdistance` are in `gensim.test.test_word2vec`. A Word2Vec model is trained on a very small corpus defined in the start of `test_word2vec`. `min_count=1` is used so that the model contains some words even though the corpus is small. `seed=42` and `workers=1` is used so that the results don't change each time you run the test (although it may be different on different machines).

The following tests are in place:

* testNonzero:
    * Compute the WMD distance between some sentences and check that the distance is not zero. This is basically to test that WMD is working. It is not a good idea to test that the distance equals some specific value, as the result may vary.
* testSymmetry:
    * Test that the the distance between sentence 1 and 2 is the same as between sentence 2 and 1.
* testIdenticalSentences:
    * Test that the distance between a sentence and itself is zero.
* WCD:
    * All the same tests as above for WCD.
* RWMD:
    * All the same tests as above for RWMD.
* testOrderWCD:
    * Check that WCD is less than WMD, as it is defined to be a lower bound.
* testOrderRWMD:
    * Check that RWMD is less than WMD, as it is defined to be a lower bound.

Note that it does not necessarily hold that WCD is less than RWMD, although it is the case a large portion of the time.

### Testing WmdSimilarity

There are tests in place in `gensim.test.test_similarities` that test all the similarity classes. Some of these, however, do not work for `WmdSimilarity`. These are therefore overwritten.

There is one test added to `WmdSimilarity` that isn't tested for the other similarity classes. This is the `testNonIncreasing` test, which (as the name suggests) checks that the similarity vector is non-increasing. It does not need to be strictly decreasing.

The tests of `WmdSimilarity` are implemented in a way very similar to the other similarity class tests.

## Prefetch and prune

The prefetch and prune algorithm is supposed to speed up the corpus based similarity computation. At the moment, it is not faster, which is why it is excluded from the first release of WMD.

Most likely, the primary problem is that the Python code that computes the distance matrix, RWMD and WCD is too slow in comparison to the fast C++ code that computes WMD. Therefore, what will most likely give the most speed-up is to convert as much as possible of the `wmdistance` code to **Cython** code.

Another problem is that I didn't experience the level of pruning claimed in the paper [1]. Pruning means discarding a document as a potential candidate without having to compute the full WMD distance. According to the paper, for some data sets they are able to prune up to 95%. In the data set I tested I experienced about 55% pruning.

### Pre-computing distances

In an attempt to mend this problem, I have tried to precompute all the distances in the vocabulary. The distances are computed in `gensim.models.word2vec.init_distances`. This makes prefetch and prune significantly faster.

A better way to do this is by *memoization*, computing distances between pairs of words when they are seen for the first time, and adding them to the pre-computed distance dictionary.

Pre-computing the distances can take up a lot of memory, use with caution.

As mentioned in the WMD tutorial [3], `gensim.models.word2vec.init_sims` is used used to normalize the word embeddings. `init_sims` must be called *before* `init_distances`. If the distances are pre-computed, and then the word vectors are normalized, the distances will be wrong.

## Pure Python WMD

As mentioned earlier, this WMD implementation relies on a C++ implementation of EMD (through the `pyemd` Python wrapper). 

It was considered in the beginning to make a brute force algorithm, purely in Python. It turned out not to be possible to make such a "brute force" algorithm; to compute the WMD distance, you have to solve the *transportation problem* (explained in [1]).

The transportation problem can be formulated as a linear program, and as such it should be possible to solve it using SciPy's linear programming solver.

## References

1. Matt J. Kusner et al., *From Word Embeddings To Document Distances*, 2015.
* WMD Gensim branch, https://github.com/olavurmortensen/gensim/tree/word_movers_distance.
* WMD Gensim tutorial, https://github.com/olavurmortensen/gensim/blob/word_movers_distance/docs/notebooks/WMD_tutorial.ipynb.